# Checking GPU

In [12]:
!nvidia-smi

Mon Feb  2 19:44:03 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Installing Libraries

In [13]:
!pip install -q ultralytics opencv-python numpy pandas

In [14]:
import cv2
import numpy as np
import pandas as pd
from collections import defaultdict
from ultralytics import YOLO

# Loading video and model

In [15]:
VIDEO_PATH = "/content/traffic.mov"
model = YOLO("yolov8n.pt")

# DEFINE COUNTING LINE

In [16]:
LINE_Y = 400

def crossed_line(prev_y, curr_y, line_y):
    return prev_y < line_y and curr_y >= line_y

# CLASS MAPPING

In [17]:
CLASS_NAMES = {
    2: "car",
    3: "motorcycle",
    5: "bus",
    7: "truck"
}


# INITIALIZE TRACKING & COUNT STORAGE

In [18]:
track_history = {}
counted_ids = set()

counts = defaultdict(int)
time_records = []

# Main tracking and counting

In [19]:
cap = cv2.VideoCapture(VIDEO_PATH)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_idx = 0


while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_idx += 1
    timestamp_sec = frame_idx / fps

    cv2.line(frame, (0, LINE_Y), (frame.shape[1], LINE_Y), (0, 0, 255), 2)

    results = model.track(
        frame,
        persist=True,
        conf=0.4,
        classes=list(CLASS_NAMES.keys()),
        tracker="bytetrack.yaml"
    )

    if results[0].boxes.id is None:
        continue

    boxes = results[0].boxes.xyxy.cpu().numpy()
    ids = results[0].boxes.id.cpu().numpy().astype(int)
    clss = results[0].boxes.cls.cpu().numpy().astype(int)

    for box, track_id, cls in zip(boxes, ids, clss):
      x1, y1, x2, y2 = map(int, box)
      cy = (y1 + y2) // 2

      if track_id not in counted_ids and cy > LINE_Y:
          counted_ids.add(track_id)

          vehicle_name = model.names[int(cls)]
          time_sec = frame_idx / fps

          time_records.append({
              "time_sec": time_sec,
              "vehicle": vehicle_name
          })

      track_history[track_id] = cy



cap.release()


requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 2 packages in 187ms
Prepared 1 package in 51ms
Installed 1 package in 3ms
 + lap==0.5.12

requirements: AutoUpdate success ✅ 0.7s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


0: 384x640 10 cars, 79.9ms
Speed: 9.2ms preprocess, 79.9ms inference, 122.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 17.4ms
Speed: 8.0ms preprocess, 17.4ms inference, 5.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 19.9ms
Speed: 8.1ms preprocess, 19.9ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 cars, 20.6ms
Speed: 9.9ms preprocess, 20.6ms inference, 3.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 8.6ms
Speed: 7.3ms preprocess, 8.6ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 car

In [20]:
import pandas as pd

df = pd.DataFrame(time_records)

if df.empty:
    print("No vehicles counted")
else:
    total_counts = df["vehicle"].value_counts()
    print("Vehicle Counts:")
    print(total_counts)


Vehicle Counts:
vehicle
car      67
truck     8
Name: count, dtype: int64


# REATE TIME-BASED CSV

In [21]:
import pandas as pd

if len(time_records) == 0:
    print("⚠️ No vehicles crossed the line. Adjust LINE_Y.")
else:
    df = pd.DataFrame(time_records)
    df["minute"] = (df["time_sec"] // 60).astype(int)

    summary = (
        df.groupby(["minute", "vehicle"])
          .size()
          .reset_index(name="count")
    )

    summary.to_csv("traffic_counts_per_minute.csv", index=False)
    summary


# SAVE CSV OUTPUT

In [22]:
CSV_PATH = "traffic_counts_per_minute.csv"
summary.to_csv(CSV_PATH, index=False)

print("Saved:", CSV_PATH)


Saved: traffic_counts_per_minute.csv


# OUTPUT VIDEO WITH COUNTING

In [23]:
cap = cv2.VideoCapture(VIDEO_PATH)

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*"XVID")
out = cv2.VideoWriter("traffic_count_output.avi", fourcc, fps, (width, height))


In [24]:
counted_ids = set()
track_history = {}
time_records = []

live_counts = {"car": 0, "truck": 0, "bus": 0, "motorcycle": 0}
frame_idx = 0


In [25]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_idx += 1

    results = model.track(
        frame,
        persist=True,
        conf=0.4,
        classes=[2, 3, 5, 7],
        tracker="bytetrack.yaml"
    )

    # Draw counting line
    cv2.line(frame, (0, LINE_Y), (width, LINE_Y), (0, 0, 255), 2)

    if results[0].boxes.id is not None:
        boxes = results[0].boxes.xyxy.cpu().numpy()
        ids   = results[0].boxes.id.cpu().numpy().astype(int)
        clss  = results[0].boxes.cls.cpu().numpy().astype(int)

        for box, track_id, cls in zip(boxes, ids, clss):
            x1, y1, x2, y2 = map(int, box)
            cy = (y1 + y2) // 2

            label = model.names[int(cls)]

            # Draw bounding box + ID
            cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
            cv2.putText(
                frame,
                f"{label} ID:{track_id}",
                (x1, y1-5),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0,255,0),
                2
            )

            if track_id not in counted_ids and cy > LINE_Y:
                counted_ids.add(track_id)
                live_counts[label] += 1

    # Draw live counts on screen
    y_offset = 30
    for k, v in live_counts.items():
        cv2.putText(
            frame,
            f"{k}: {v}",
            (10, y_offset),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            (255,255,0),
            2
        )
        y_offset += 25

    out.write(frame)

cap.release()
out.release()



0: 384x640 2 cars, 8.0ms
Speed: 3.9ms preprocess, 8.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 8.4ms
Speed: 3.8ms preprocess, 8.4ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 12.3ms
Speed: 4.3ms preprocess, 12.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 9.1ms
Speed: 3.2ms preprocess, 9.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 17.9ms
Speed: 3.8ms preprocess, 17.9ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 19.9ms
Speed: 4.2ms preprocess, 19.9ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 cars, 21.0ms
Speed: 8.0ms preprocess, 21.0ms inference, 4.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 20.8ms
Speed: 3.9ms preprocess, 20.8ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384

In [ ]:
from IPython.display import Video
Video("traffic_count_output.avi", embed=True)
